In [14]:
import cx_Oracle
import pandas as pd
import hashlib


In [28]:

# Connect to Oracle XE
try:
    conn = cx_Oracle.connect("system/Testing1@localhost/XEPDB1")
    cursor = conn.cursor()
    print(" Connected to Oracle XE successfully.")
except cx_Oracle.DatabaseError as e:
    print("Connection failed:", e)
    exit()

# Step 1: Insert banks — run only once
banks = {
    'com.combanketh.mobilebanking': 'CBE',
    'com.boa.boaMobileBanking': 'BOA',
    'com.cr2.amolelight': 'Dashen'
}

try:
    for app_id, name in banks.items():
        cursor.execute("INSERT INTO banks (bank_name, app_id) VALUES (:1, :2)", (name, app_id))
    conn.commit()
    print(" Inserted banks successfully.")
except cx_Oracle.DatabaseError as e:
    print(" Skipping bank insertion (possibly already inserted):", e)

# Step 2: Fetch bank_id mapping
cursor.execute("SELECT bank_id, bank_name FROM banks")
bank_id_map = {name: bank_id for bank_id, name in cursor.fetchall()}
print(" Fetched bank ID mappings:", bank_id_map)

# Step 3: Load cleaned review data
df = pd.read_csv("../data/review_with_themes.csv")  # Update path if needed

# Ensure necessary columns exist
required_columns = ['review', 'rating', 'date', 'bank', 'sentiment', 'theme']
if not all(col in df.columns for col in required_columns):
    missing = [col for col in required_columns if col not in df.columns]
    raise ValueError(f"Missing required columns in DataFrame: {missing}")

# Step 4: Insert reviews
# Fetch bank_id mapping
cursor.execute("SELECT bank_id, bank_name FROM banks")
bank_id_map = {name: bank_id for bank_id, name in cursor.fetchall()}

for _, row in df.iterrows():
    try:
        review_id = hashlib.md5(row['review'].encode('utf-8')).hexdigest()
        bank_id = bank_id_map.get(row['bank'])
        review_text = row['review']
        rating = int(row['rating'])
        review_date = str(row['date'])[:10]
        sentiment = row['sentiment']
        theme = row.get('theme', None)

        cursor.execute("""
            MERGE INTO reviews r
            USING (SELECT :1 AS review_id FROM dual) src
            ON (r.review_id = src.review_id)
            WHEN NOT MATCHED THEN
                INSERT (
                    review_text, rating, review_date,bank_id, sentiment, theme
                )
                VALUES (
                         :1, :2, TO_DATE(:3, 'YYYY-MM-DD'), :4, :5, :6
                )
        """, (review_id, bank_id, review_text, rating, review_date, sentiment, theme))
    except Exception as e:
        print(f"Failed to insert review: {e}")

conn.commit()
print(f"successfully inserted")
cursor.close()
conn.close()


 Connected to Oracle XE successfully.
 Inserted banks successfully.
 Fetched bank ID mappings: {'CBE': 74, 'BOA': 75, 'Dashen': 76}
successfully inserted


In [32]:
cursor.execute("SELECT * FROM reviews")

for row in cursor.fetchall():
    print(row)


InterfaceError: not open